Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from tqdm import tqdm
from heapq import heappush, heappop
from itertools import count
import numpy as np
from random import choice
from collections import namedtuple

### Teacher initialization

In [2]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])
action_cost = 1

def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal = state
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 259446.73it/s]


array([[0, 6, 4],
       [8, 3, 5],
       [2, 1, 7]])

### A* problem solving

In [3]:
#HashClass to get a better computation time (seen on the student that did the presentation in class)
class HashClass:
    def __init__(self, state: np.ndarray):
        self.state = state
        self.hash = hash(state.tobytes())

    def __hash__(self):
        return self.hash

    def __eq__(self, other):
        return np.array_equal(self.state, other.state)

#Defining the final goal state of the puzzle (1 then 2 and so on)
goal_positions = {value: divmod(value - 1, PUZZLE_DIM) for value in range(1, PUZZLE_DIM**2)}

#Heuristics function (using the distance between each element and their goal position)
def heuristics(state: np.ndarray) -> int:
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  
                target_x, target_y = goal_positions[value]
                distance += abs(x-target_x) + abs(y-target_y)
    return distance

#A* path finding algorithm
def a_star(initial_state: np.ndarray, goal_state: np.ndarray):
    frontier = [] #Priority queue of visiting states initializasion
    visited = set()
    unique_id = count()  #I use here a unique counter for the states for my heap, enabeling better computation time
    heappush(frontier, (0, next(unique_id), HashClass(initial_state), []))  #Here I add an empty element to the queue

    #Progress bar to track algorithm since it is slow on large numbers by calculating percentage of well-places pieces
    progress = tqdm(total=100, desc="Progress in solving", position=0, leave=True, dynamic_ncols=True)

    max_correct_tiles = PUZZLE_DIM * PUZZLE_DIM  
    last_percentage = 0  

    while frontier:

        f, _, current_state, path = heappop(frontier) #I extract the state with the lowest total estimated cost (f) from the priority queue

        #Progress update, only if the percentage increases to avoid mini movmennts
        correct_tiles = np.sum(current_state.state==goal_state)
        percentage_correct = (correct_tiles/max_correct_tiles)*100
        if percentage_correct > last_percentage:
            progress.update(percentage_correct-last_percentage)
            last_percentage = percentage_correct

        if np.array_equal(current_state.state, goal_state):
            total_cost = len(path)  # Coût total = nombre d'actions
            return path, total_cost

        visited.add(current_state)

        for act in available_actions(current_state.state):
            new_state = HashClass(do_action(current_state.state, act))
            if new_state in visited:
                continue

            g = len(path) + action_cost  
            h = heuristics(new_state.state)  
            f = g + h  #cst function that is supposed to be admissible and consistent, with cumulated cost and estimated one

            heappush(frontier, (f, next(unique_id), new_state, path + [act])) #Heappush enables to make sure that the less costly state 
                                                                                # is put first in the visiting order
    return None, None

#Puzzle solving
solution, total_cost = a_star(state, goal)

#Results
print(f"Puzzle solved in {len(solution)} actions with a cost of {total_cost}.")
solved_state = state.copy()
for act in solution:
    solved_state = do_action(solved_state, act)

#Vizualise final state
print("Final puzzle")
print(solved_state)


Progress in solving: 100%|██████████| 100.0/100 [00:00<00:00, 2822.28it/s]

Puzzle solved in 24 actions with a cost of 24.
Final puzzle
[[1 2 3]
 [4 5 6]
 [7 8 0]]
